In [1]:
import subprocess, sys, shutil
from pathlib import Path

# --- 1) Locate or auto-install an ffmpeg binary we can call ---
def get_ffmpeg_path():
    # a) Already on PATH?
    p = shutil.which("ffmpeg")
    if p:
        return Path(p)

    # b) Try imageio-ffmpeg (downloads a local static ffmpeg if missing)
    try:
        import imageio_ffmpeg
    except ModuleNotFoundError:
        print("Installing imageio-ffmpeg (one-time)…")
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", "imageio-ffmpeg"], check=True)
        import imageio_ffmpeg

    return Path(imageio_ffmpeg.get_ffmpeg_exe())

ffmpeg_path = get_ffmpeg_path()

# Optional: show version (helps confirm it's working)
try:
    out = subprocess.check_output([str(ffmpeg_path), "-version"], text=True, stderr=subprocess.STDOUT)
    print(out.splitlines()[0])
except Exception as e:
    raise RuntimeError(f"ffmpeg exists at {ffmpeg_path} but couldn't run it: {e}")

# --- 2) Batch convert settings ---
input_folder  = Path("Videos")              # <-- your mixed videos: rock01.mp4, paper00.mp4, ...
output_root   = Path("videos_transformed")  # <-- root; we'll add subfolders Rock/Paper/Scissor
output_root.mkdir(exist_ok=True)

CRF    = "23"       # 18–24 common range; lower = better quality, larger files
PRESET = "medium"   # ultrafast..placebo (slower = smaller file)
FPS    = "20"

videos = sorted(input_folder.glob("*.mp4"))
if not videos:
    print("⚠️ No .mp4 files found in 'Videos'. Make sure the folder sits next to this notebook.")
else:
    for src in videos:
        # --- NEW: infer class from filename prefix ---
        stem_lower = src.stem.lower()
        if stem_lower.startswith("rock"):
            cls = "Rock"
        elif stem_lower.startswith("paper"):
            cls = "Paper"
        elif stem_lower.startswith("scissor"):
            cls = "Scissor"   # covers scissor / scissors etc. because of startswith
        else:
            print(f"⚠️ Skipping (could not infer class): {src.name}")
            continue

        # each class gets its own subfolder
        class_folder = output_root / cls
        class_folder.mkdir(parents=True, exist_ok=True)

        dst = class_folder / f"{src.stem}_720p20fps.mp4"
        print(f"[{cls}] Processing: {src.name}")

        # Fit inside 720p without stretching; keep mp4 (H.264 + AAC), 20 fps
        cmd = [
            str(ffmpeg_path),
            "-y",
            "-i", str(src),
            "-vf", f"scale=-2:720:force_original_aspect_ratio=decrease,fps={FPS}",
            "-r", FPS,
            "-c:v", "libx264",
            "-preset", PRESET,
            "-crf", CRF,
            "-pix_fmt", "yuv420p",
            "-c:a", "aac",
            "-b:a", "160k",
            str(dst)
        ]

        # Run and stream output so you can see progress in Jupyter
        proc = subprocess.run(cmd, text=True)
        if proc.returncode == 0:
            print(f"✅ Saved: {dst}\n")
        else:
            print(f"❌ Failed: {src.name}\n")

    print("🎬 Done.")


ffmpeg version 7.1-essentials_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
[Paper] Processing: paper00.mp4
✅ Saved: videos_transformed\Paper\paper00_720p20fps.mp4

[Paper] Processing: paper01.mp4
✅ Saved: videos_transformed\Paper\paper01_720p20fps.mp4

[Paper] Processing: paper02.mp4
✅ Saved: videos_transformed\Paper\paper02_720p20fps.mp4

[Paper] Processing: paper03.mp4
✅ Saved: videos_transformed\Paper\paper03_720p20fps.mp4

[Rock] Processing: rock00.mp4
✅ Saved: videos_transformed\Rock\rock00_720p20fps.mp4

[Rock] Processing: rock01.mp4
✅ Saved: videos_transformed\Rock\rock01_720p20fps.mp4

[Rock] Processing: rock02.mp4
✅ Saved: videos_transformed\Rock\rock02_720p20fps.mp4

[Rock] Processing: rock03.mp4
✅ Saved: videos_transformed\Rock\rock03_720p20fps.mp4

[Rock] Processing: rock04.mp4
✅ Saved: videos_transformed\Rock\rock04_720p20fps.mp4

[Rock] Processing: rock06.mp4
✅ Saved: videos_transformed\Rock\rock06_720p20fps.mp4

[Scissor] Processing: scissors00.mp4
✅ 

In [2]:
# %%
import subprocess
from pathlib import Path
import shutil

# === Locate ffmpeg again (reuse imageio-ffmpeg if needed) ===
def get_ffmpeg_path():
    p = shutil.which("ffmpeg")
    if p:
        return Path(p)
    import imageio_ffmpeg
    return Path(imageio_ffmpeg.get_ffmpeg_exe())

ffmpeg_path = get_ffmpeg_path()

transformed_root = Path("videos_transformed")
output_root      = Path("videos_noaudio")
output_root.mkdir(exist_ok=True)

classes = ["Rock", "Paper", "Scissor"]  # adjust if your folder names differ

def strip_audio(src: Path, dst: Path):
    """Try fast copy first; if it fails, re-encode video without audio."""
    print(f"  → Fast path: copy video, drop audio")
    cmd_copy = [
        str(ffmpeg_path),
        "-y",
        "-i", str(src),
        "-an",          # remove all audio streams
        "-c:v", "copy", # keep video stream unchanged
        str(dst)
    ]
    proc = subprocess.run(cmd_copy, capture_output=True, text=True)

    if proc.returncode == 0:
        print(f"    ✅ Success (copy).")
        return True

    # If we get here, copy failed
    print(f"    ⚠️ Copy failed, will retry with re-encode.")
    # Optional: show a short snippet of the error
    print("    ffmpeg said (copy):", proc.stderr.splitlines()[-1])

    # Fallback: re-encode video (slower, but more robust across weird codecs)
    cmd_reencode = [
        str(ffmpeg_path),
        "-y",
        "-i", str(src),
        "-an",             # remove audio
        "-c:v", "libx264", # re-encode
        "-preset", "medium",
        "-crf", "23",
        "-pix_fmt", "yuv420p",
        str(dst)
    ]
    proc2 = subprocess.run(cmd_reencode, capture_output=True, text=True)

    if proc2.returncode == 0:
        print(f"    ✅ Success (re-encoded).")
        return True

    # Still failed – report error and give up on this file
    print(f"    ❌ Re-encode also failed for {src.name}")
    print("    ffmpeg said (re-encode):", proc2.stderr.splitlines()[-1])
    return False


for cls in classes:
    input_folder  = transformed_root / cls
    output_folder = output_root / cls
    output_folder.mkdir(parents=True, exist_ok=True)

    videos = sorted(input_folder.glob("*.mp4"))
    if not videos:
        print(f"⚠️ No .mp4 videos found in '{input_folder}'.")
        continue

    for src in videos:
        dst = output_folder / f"{src.stem}_nosound.mp4"
        print(f"[{cls}] Removing sound from: {src.name}")
        ok = strip_audio(src, dst)
        if ok:
            print(f"✅ Saved silent video: {dst}\n")
        else:
            print(f"❌ Skipping {src.name} due to repeated ffmpeg failure.\n")

print("🎬 All videos processed successfully — no audio now!")


[Rock] Removing sound from: rock00_720p20fps.mp4
  → Fast path: copy video, drop audio
    ✅ Success (copy).
✅ Saved silent video: videos_noaudio\Rock\rock00_720p20fps_nosound.mp4

[Rock] Removing sound from: rock01_720p20fps.mp4
  → Fast path: copy video, drop audio
    ✅ Success (copy).
✅ Saved silent video: videos_noaudio\Rock\rock01_720p20fps_nosound.mp4

[Rock] Removing sound from: rock02_720p20fps.mp4
  → Fast path: copy video, drop audio
    ✅ Success (copy).
✅ Saved silent video: videos_noaudio\Rock\rock02_720p20fps_nosound.mp4

[Rock] Removing sound from: rock03_720p20fps.mp4
  → Fast path: copy video, drop audio
    ✅ Success (copy).
✅ Saved silent video: videos_noaudio\Rock\rock03_720p20fps_nosound.mp4

[Rock] Removing sound from: rock04_720p20fps.mp4
  → Fast path: copy video, drop audio
    ✅ Success (copy).
✅ Saved silent video: videos_noaudio\Rock\rock04_720p20fps_nosound.mp4

[Rock] Removing sound from: Rock05_720p20fps.mp4
  → Fast path: copy video, drop audio
    ✅ Su

In [3]:
import subprocess, shutil
from pathlib import Path

# -------- Locate ffmpeg (PATH or imageio-ffmpeg fallback) --------
def get_ffmpeg_path():
    p = shutil.which("ffmpeg")
    if p:
        return Path(p)
    import imageio_ffmpeg
    return Path(imageio_ffmpeg.get_ffmpeg_exe())

ffmpeg = get_ffmpeg_path()

# -------- Paths & categories --------
base = Path("videos_noaudio")
categories = ["Rock", "Paper", "scissor"]  # use your exact folder names

# -------- Extraction settings --------
# Output image format: jpg (you can switch to png if you prefer)
# JPEG quality: -q:v 2 is high quality (2 best .. 31 worst)
jpg_quality = "2"

any_found = False
for cat in categories:
    folder = base / cat
    if not folder.exists():
        print(f"⚠️ Folder not found: {folder}")
        continue

    videos = sorted(folder.glob("*.mp4"))
    if not videos:
        print(f"⚠️ No .mp4 files in {folder}")
        continue

    any_found = True
    for src in videos:
        # Frame file pattern: <video_stem>_001.jpg, <video_stem>_002.jpg, ...
        out_pattern = str(folder / f"{src.stem}_%03d.jpg")
        print(f"Extracting frames from: {src.relative_to(base)}")

        cmd = [
            str(ffmpeg),
            "-y",
            "-i", str(src),
            "-start_number", "1",
            "-q:v", jpg_quality,   # JPEG quality (lower is better)
            out_pattern
        ]

        # Run ffmpeg; this extracts every frame at the video's native fps
        proc = subprocess.run(cmd)
        if proc.returncode == 0:
            print(f"✅ Frames saved as: {src.stem}_001.jpg, {src.stem}_002.jpg, ... in {folder}\n")
        else:
            print(f"❌ Failed on {src}\n")

if not any_found:
    print("No videos found to process. Check your folder names and contents.")
else:
    print("🎬 Done extracting frames.")


Extracting frames from: Rock\rock00_720p20fps_nosound.mp4
✅ Frames saved as: rock00_720p20fps_nosound_001.jpg, rock00_720p20fps_nosound_002.jpg, ... in videos_noaudio\Rock

Extracting frames from: Rock\rock01_720p20fps_nosound.mp4
✅ Frames saved as: rock01_720p20fps_nosound_001.jpg, rock01_720p20fps_nosound_002.jpg, ... in videos_noaudio\Rock

Extracting frames from: Rock\rock02_720p20fps_nosound.mp4
✅ Frames saved as: rock02_720p20fps_nosound_001.jpg, rock02_720p20fps_nosound_002.jpg, ... in videos_noaudio\Rock

Extracting frames from: Rock\rock03_720p20fps_nosound.mp4
✅ Frames saved as: rock03_720p20fps_nosound_001.jpg, rock03_720p20fps_nosound_002.jpg, ... in videos_noaudio\Rock

Extracting frames from: Rock\rock04_720p20fps_nosound.mp4
✅ Frames saved as: rock04_720p20fps_nosound_001.jpg, rock04_720p20fps_nosound_002.jpg, ... in videos_noaudio\Rock

Extracting frames from: Rock\Rock05_720p20fps_nosound.mp4
✅ Frames saved as: Rock05_720p20fps_nosound_001.jpg, Rock05_720p20fps_nosound